# Objetivo_2

In [65]:
import pandas as pd
import numpy as np
import plotly.express as px
import altair as alt
import datos
import plotly.graph_objects as go
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

tipo de afiliación, lugar de nacimiento/procedencia, escolaridad,   antecedente personal de dislipidemia o hipercolesterolemia, de hipertensión, de infarto de miocardio de diabetes, insuficiencia cardiaca congestiva, nefropatía, arritmia, válvula, angeopatía, fumador y bebedor.  

+ IMC_calculada
+ perimetro_abdominal
+ presion_arterial_sistolica
+ presion_arterial_diastolica

+ <19.9 bajo peso, 20-24.9 normo peso, 25-29.9 sobrepeso, >=30 obesidad

+ normal (diastólica <80 y sistólica <120), 
 elevada (sistólica 120-129 y diastólica <80),
 alta nivel 1 (sistólica 130-139 o diastólica 80-89), 
 alta nivel 2 (sistólica >=140, diastólica >=90), 
 crisis hipertensiva (>180 diastólica y/o >120 sistólica)
+ aumentado >94 hombres, >80 mujeres; incremento sustancial >102 hombres y >88 en mujeres

### IMC

In [66]:

intervalos = {
    'IMC': [0, 19.9, 24.9, 29.9, 40, max(datos.datos['IMC'])+1]
}

# Agrupaciones disponibles
agrupaciones = [
    'sexo',
    'tipo_de_afiliacion',
    'lugar_de_procedencia',
    'lugar_de_nacimiento',
    'escolaridad',
    'diabetes mellitus',
    'hipercolesterolemia',
    'dislipidemia'
]
def plot_all():
    # Graficar IMC crudo
    fig = px.box(datos.datos, y='IMC', title='Boxplot crudo del IMC')
    fig.update_layout(xaxis_title='IMC')
    fig.show()

    fig = px.histogram(datos.datos, x='IMC', nbins=10, title='Histograma crudo del IMC', marginal='rug')
    fig.update_layout(xaxis_title='IMC')
    fig.show()

    # Mostrar tabla descriptiva general
    tabla = datos.datos['IMC'].describe().reset_index()
    tabla.columns = ['Estadística', 'Valor']
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla[col] for col in tabla.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas del IMC')
    fig.show()

    # Graficar IMC por cada agrupación, excluyendo 'lugar_de_procedencia' y 'lugar_de_nacimiento'
    for agrupacion in agrupaciones:
        if agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            continue

        # Estadísticas descriptivas por agrupación
        tabla2 = datos.datos.groupby(agrupacion)['IMC'].describe().reset_index()
        tabla2 = tabla2.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
        fig = go.Figure(data=[go.Table(
            header=dict(values=list(tabla2.columns), fill_color='paleturquoise', align='left'),
            cells=dict(values=[tabla2[col] for col in tabla2.columns], fill_color='lavender', align='left')
        )])
        fig.update_layout(title_text=f'Estadísticas Descriptivas del IMC por {agrupacion}')
        fig.show()

        # Boxplot y histograma por agrupación
        fig = px.box(datos.datos, x='IMC', y=agrupacion, title=f'Boxplot del IMC por {agrupacion}')
        fig.update_layout(xaxis_title='IMC')
        fig.show()

        fig = px.histogram(datos.datos, x='IMC', color=agrupacion, nbins=10, title=f'Histograma del IMC por {agrupacion}', marginal='rug')
        fig.update_layout(xaxis_title='IMC')
        fig.show()


    
    datos.datos['IMC_intervalo'] = pd.cut(datos.datos['IMC'], bins=[0, 19.9, 24.9, 29.9, 40.0, 50.66])
    datos.datos['IMC_intervalo'] = datos.datos['IMC_intervalo'].astype(str)

        
    tabla3 = datos.datos.groupby('IMC_intervalo')['IMC'].describe().reset_index()
    tabla3 = tabla3.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla3.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla3[col] for col in tabla3.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas del IMC por Intervalos')
    fig.show()

    orden_intervalos = ['(0.0, 19.9]', '(19.9, 24.9]', '(24.9, 29.9]', '(29.9, 40.0]', '(40.0, 50.66]']
    fig = px.box(datos.datos, x=pd.Categorical(datos.datos['IMC_intervalo'], categories=orden_intervalos, ordered=True), 
                y='IMC', title="Boxplot del IMC por Intervalos", color_discrete_sequence=["blue"])
    fig.update_layout(xaxis_title='Intervalo IMC', yaxis_title='IMC', xaxis={'categoryorder':'array', 'categoryarray':orden_intervalos})
    fig.show()
# Llamar a la función para generar todas las gráficas
plot_all()

# Mostrar solo tablas de frecuencias para lugar_de_procedencia y lugar_de_nacimiento
for agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento']:
    tabla_freq = datos.datos[agrupacion].value_counts().reset_index()
    tabla_freq.columns = [agrupacion, 'Frecuencia']
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_freq.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_freq[col] for col in tabla_freq.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text=f'Tabla de Frecuencias para {agrupacion}')
    fig.show()


### Perimetro abdominal

In [67]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Definir los intervalos para perimetro_abdominal
intervalos = {
    'masculino': [0, 94, 102, max(datos.datos['perimetro_abdominal'])],
    'femenino': [0, 80, 88, max(datos.datos['perimetro_abdominal'])] 
}
agrupaciones = [
        'sexo', 'tipo_de_afiliacion', 'lugar_de_procedencia',
        'lugar_de_nacimiento', 'escolaridad', 'diabetes mellitus',
        'hipercolesterolemia', 'dislipidemia'
    ]

def plot_all():
    # Graficar perimetro_abdominal crudo
    fig = px.box(datos.datos, y='perimetro_abdominal', title='Boxplot crudo de perimetro_abdominal')
    fig.update_layout(yaxis_title='Perímetro Abdominal')
    fig.show()

    fig = px.histogram(datos.datos, x='perimetro_abdominal', nbins=10, title='Histograma crudo de perimetro_abdominal', marginal='rug')
    fig.update_layout(xaxis_title='Perímetro Abdominal')
    fig.show()

    # Mostrar tabla descriptiva general verticalmente
    tabla = datos.datos['perimetro_abdominal'].describe().reset_index()
    tabla.columns = ['Estadística', 'Valor']
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla[col] for col in tabla.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas del perimetro_abdominal')
    fig.show()

    # Agrupaciones
    agrupaciones = [
        'sexo', 'tipo_de_afiliacion', 'lugar_de_procedencia',
        'lugar_de_nacimiento', 'escolaridad', 'diabetes mellitus',
        'hipercolesterolemia', 'dislipidemia'
    ]

    for agrupacion in agrupaciones:
        if agrupacion not in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            # Graficar perimetro_abdominal por la variable de agrupación
            fig = px.box(datos.datos, x='perimetro_abdominal', y=agrupacion, title=f'Boxplot de perimetro_abdominal por {agrupacion}')
            fig.update_layout(xaxis_title='Perímetro Abdominal')
            fig.show()

            fig = px.histogram(datos.datos, x='perimetro_abdominal', color=agrupacion, nbins=10, title=f'Histograma de perimetro_abdominal por {agrupacion}', marginal='rug')
            fig.update_layout(xaxis_title='Perímetro Abdominal')
            fig.show()

            # Mostrar tabla descriptiva por grupo verticalmente
            tabla2 = datos.datos.groupby(agrupacion)['perimetro_abdominal'].describe().reset_index()
            tabla2 = tabla2.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
            fig = go.Figure(data=[go.Table(
                header=dict(values=list(tabla2.columns), fill_color='paleturquoise', align='left'),
                cells=dict(values=[tabla2[col] for col in tabla2.columns], fill_color='lavender', align='left')
            )])
            fig.update_layout(title_text=f'Estadísticas Descriptivas del perimetro_abdominal por {agrupacion}')
            fig.show()



# Llamar a la función para generar todas las gráficas
plot_all()



### POC_hba1

In [68]:

# Definir los intervalos para POC_hba1c(%)
intervalos = {
    'POC_hba1c(%)': [0, 5.7, 6.4, 7.5, 10, max(datos.datos['POC_hba1c(%)']) + 4]  # Ajusta los intervalos según sea necesario
}

def plot_all():
    # Graficar POC_hba1c(%) crudo
    fig = px.box(datos.datos, y='POC_hba1c(%)', title='Boxplot crudo de POC_hba1c(%)')
    fig.update_layout(yaxis_title='POC_hba1c(%)')
    fig.show()

    fig = px.histogram(datos.datos, x='POC_hba1c(%)', nbins=10, title='Histograma crudo de POC_hba1c(%)', marginal='rug')
    fig.update_layout(xaxis_title='POC_hba1c(%)')
    fig.show()

    # Mostrar tabla descriptiva general verticalmente
    tabla = datos.datos['POC_hba1c(%)'].describe().reset_index()
    tabla.columns = ['Estadística', 'Valor']
    tabla_vertical = tabla.T  # Transponer para mostrar verticalmente
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_vertical.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_vertical[col] for col in tabla_vertical.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas de POC_hba1c(%)')
    fig.show()

    # Agrupaciones
    agrupaciones = [
        'sexo', 'tipo_de_afiliacion', 'lugar_de_procedencia',
        'lugar_de_nacimiento', 'escolaridad', 'diabetes mellitus',
        'hipercolesterolemia', 'dislipidemia'
    ]

    for agrupacion in agrupaciones:
        if agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            pass
        else:
            fig = px.box(datos.datos, x='POC_hba1c(%)', y=agrupacion, title=f'Boxplot de POC_hba1c(%) por {agrupacion}')
            fig.update_layout(xaxis_title='POC_hba1c(%)', yaxis_title=agrupacion)
            fig.show()

            fig = px.histogram(datos.datos, x='POC_hba1c(%)', color=agrupacion, nbins=10, title=f'Histograma de POC_hba1c(%) por {agrupacion}', marginal='rug')
            fig.update_layout(xaxis_title='POC_hba1c(%)')
            fig.show()

            # Mostrar tabla descriptiva por grupo verticalmente
            tabla_agrupacion = datos.datos.groupby(agrupacion)['POC_hba1c(%)'].describe().reset_index()
            tabla_agrupacion = tabla_agrupacion.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
            tabla_agrupacion = tabla_agrupacion.melt(id_vars=agrupacion, var_name='Estadística', value_name='Valor')
            fig = go.Figure(data=[go.Table(
                header=dict(values=list(tabla_agrupacion.columns), fill_color='paleturquoise', align='left'),
                cells=dict(values=[tabla_agrupacion[col] for col in tabla_agrupacion.columns], fill_color='lavender', align='left')
            )])
            fig.update_layout(title_text=f'Estadísticas Descriptivas de POC_hba1c(%) por {agrupacion}')
            fig.show()

    # Mostrar estadísticas descriptivas por intervalos
    datos.datos['POC_hba1c(%)_intervalo'] = pd.cut(datos.datos['POC_hba1c(%)'], intervalos['POC_hba1c(%)'], right=False).astype(str)
    tabla_intervalos = datos.datos.groupby('POC_hba1c(%)_intervalo')['POC_hba1c(%)'].describe().reset_index()
    tabla_intervalos.columns = ['Intervalo', 'Cuenta', 'Media', 'Desviación estándar', 'Mínimo', '25%', 'Mediana', '75%', 'Máximo']
    tabla_intervalos = tabla_intervalos.melt(id_vars='Intervalo', var_name='Estadística', value_name='Valor')
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_intervalos.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_intervalos[col] for col in tabla_intervalos.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas por Intervalos de POC_hba1c(%)')
    fig.show()

    datos.datos['POC_hba1c_intervalo'] = pd.cut(datos.datos['POC_hba1c(%)'], bins=[0, 5.7, 6.4, 7.5, 10.0, 16.3])
    datos_filtrados = datos.datos.dropna(subset=['POC_hba1c_intervalo'])
    datos_filtrados['POC_hba1c_intervalo'] = datos_filtrados['POC_hba1c_intervalo'].astype(str)
    orden_intervalos = ['(0.0, 5.7]', '(5.7, 6.4]', '(6.4, 7.5]', '(7.5, 10.0]', '(10.0, 16.3]']
    fig = px.box(datos_filtrados, 
                x=pd.Categorical(datos_filtrados['POC_hba1c_intervalo'], categories=orden_intervalos, ordered=True), 
                y='POC_hba1c(%)', 
                title="Boxplot POC_hba1c(%) por Intervalos", 
                color_discrete_sequence=["blue"])
    fig.update_layout(xaxis_title='Intervalo', yaxis_title='POC_hba1c(%)', xaxis={'categoryorder':'array', 'categoryarray':orden_intervalos})
    fig.show()
plot_all()



C:\Users\loren\AppData\Local\Temp\ipykernel_17820\1819041043.py:71: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Presion Arterial

####   Presion arterial sistolica

In [69]:
intervalos = {
    'masculino': [0, 120, 130, 139, max(datos.datos['presion_arterial_sistolica'])],
    'femenino': [0, 120, 130, 139, max(datos.datos['presion_arterial_sistolica'])]
}


def plot_all():
    # Graficar presión arterial sistólica crudo
    fig = px.box(datos.datos, y='presion_arterial_sistolica', title='Boxplot crudo de presión arterial sistólica')
    fig.update_layout(yaxis_title='Presión Arterial Sistólica')
    fig.show()

    fig = px.histogram(datos.datos, x='presion_arterial_sistolica', nbins=10, title='Histograma crudo de presión arterial sistólica', marginal='rug')
    fig.update_layout(xaxis_title='Presión Arterial Sistólica')
    fig.show()

    # Mostrar tabla descriptiva general verticalmente
    tabla = datos.datos['presion_arterial_sistolica'].describe().reset_index()
    tabla.columns = ['Estadística', 'Valor']
    tabla_vertical = tabla.T  # Transponer para mostrar verticalmente
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_vertical.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_vertical[col] for col in tabla_vertical.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas de presión arterial sistólica')
    fig.show()

    # Agrupaciones
    agrupaciones = [
        'sexo', 'tipo_de_afiliacion', 'lugar_de_procedencia',
        'lugar_de_nacimiento', 'escolaridad', 'diabetes mellitus',
        'hipercolesterolemia', 'dislipidemia'
    ]

    for agrupacion in agrupaciones:
        if agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            pass
        else:
            fig = px.box(datos.datos, x='presion_arterial_sistolica', y=agrupacion, title=f'Boxplot de presión arterial sistólica por {agrupacion}')
            fig.update_layout(xaxis_title='Presión Arterial Sistólica', yaxis_title=agrupacion)
            fig.show()

            fig = px.histogram(datos.datos, x='presion_arterial_sistolica', color=agrupacion, nbins=10, title=f'Histograma de presión arterial sistólica por {agrupacion}', marginal='rug')
            fig.update_layout(xaxis_title='Presión Arterial Sistólica')
            fig.show()

            # Mostrar tabla descriptiva por grupo verticalmente
            tabla_agrupacion = datos.datos.groupby(agrupacion)['presion_arterial_sistolica'].describe().reset_index()
            tabla_agrupacion = tabla_agrupacion.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
            tabla_agrupacion = tabla_agrupacion.melt(id_vars=agrupacion, var_name='Estadística', value_name='Valor')
            fig = go.Figure(data=[go.Table(
                header=dict(values=list(tabla_agrupacion.columns), fill_color='paleturquoise', align='left'),
                cells=dict(values=[tabla_agrupacion[col] for col in tabla_agrupacion.columns], fill_color='lavender', align='left')
            )])
            fig.update_layout(title_text=f'Estadísticas Descriptivas de presión arterial sistólica por {agrupacion}')
            fig.show()

    # Mostrar estadísticas descriptivas por intervalos
    datos.datos['presion_arterial_sistolica_intervalo'] = pd.cut(
        datos.datos['presion_arterial_sistolica'],
        intervalos['masculino'],  # Usar intervalo masculino como ejemplo
        right=False
    ).astype(str)
    tabla_intervalos = datos.datos.groupby('presion_arterial_sistolica_intervalo')['presion_arterial_sistolica'].describe().reset_index()
    tabla_intervalos.columns = ['Intervalo', 'Cuenta', 'Media', 'Desviación estándar', 'Mínimo', '25%', 'Mediana', '75%', 'Máximo']
    tabla_intervalos = tabla_intervalos.melt(id_vars='Intervalo', var_name='Estadística', value_name='Valor')
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_intervalos.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_intervalos[col] for col in tabla_intervalos.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas por Intervalos de presión arterial sistólica')
    fig.show()


    intervalos_masculino = [0, 120, 130, 139, float('inf')]
    labels = ['[0, 120)', '[120, 130)', '[130, 139)', '[139, inf)']
    datos.datos['presion_arterial_sistolica_intervalo'] = pd.cut(
        datos.datos['presion_arterial_sistolica'],
        bins=intervalos_masculino,
        labels=labels,
        right=False,
        include_lowest=True
    )
    datos_filtrados = datos.datos.dropna(subset=['presion_arterial_sistolica_intervalo'])
    datos_filtrados['presion_arterial_sistolica_intervalo'] = pd.Categorical(
        datos_filtrados['presion_arterial_sistolica_intervalo'],
        categories=labels,
        ordered=True
    )
    fig = px.box(
        datos_filtrados, 
        x='presion_arterial_sistolica_intervalo', 
        y='presion_arterial_sistolica', 
        title='Boxplot de Presión Arterial Sistólica por Intervalos', 
        color_discrete_sequence=["blue"]
    )
    fig.update_layout(
        xaxis_title='Intervalo', 
        yaxis_title='Presión Arterial Sistólica',
        xaxis={'categoryorder': 'array', 'categoryarray': labels}  
    )
    fig.show()



plot_all()



C:\Users\loren\AppData\Local\Temp\ipykernel_17820\2900655908.py:85: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Presion arterial diastolica

In [70]:
max_diastolica = max(datos.datos['presion_arterial_diastolica'])
intervalos = {
    'masculino': [0, 80, 90, 120, max_diastolica],
    'femenino': [0, 80, 90, 120, max_diastolica]
}

def plot_all():
    # Graficar presión arterial diastólica crudo
    fig = px.box(datos.datos, y='presion_arterial_diastolica', title='Boxplot crudo de presión arterial diastólica')
    fig.update_layout(yaxis_title='Presión Arterial Diastólica')
    fig.show()

    fig = px.histogram(datos.datos, x='presion_arterial_diastolica', nbins=10, title='Histograma crudo de presión arterial diastólica', marginal='rug')
    fig.update_layout(xaxis_title='Presión Arterial Diastólica')
    fig.show()

    # Mostrar tabla descriptiva general verticalmente
    tabla = datos.datos['presion_arterial_diastolica'].describe().reset_index()
    tabla.columns = ['Estadística', 'Valor']
    tabla_vertical = tabla.T  # Transponer para mostrar verticalmente
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_vertical.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_vertical[col] for col in tabla_vertical.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas de presión arterial diastólica')
    fig.show()

    # Agrupaciones
    agrupaciones = [
        'sexo', 'tipo_de_afiliacion', 'lugar_de_procedencia',
        'lugar_de_nacimiento', 'escolaridad', 'diabetes mellitus',
        'hipercolesterolemia', 'dislipidemia'
    ]

    for agrupacion in agrupaciones:
        if agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            pass
        else:
            fig = px.box(datos.datos, x='presion_arterial_diastolica', y=agrupacion, title=f'Boxplot de presión arterial diastólica por {agrupacion}')
            fig.update_layout(xaxis_title='Presión Arterial Diastólica', yaxis_title=agrupacion)
            fig.show()

            fig = px.histogram(datos.datos, x='presion_arterial_diastolica', color=agrupacion, nbins=10, title=f'Histograma de presión arterial diastólica por {agrupacion}', marginal='rug')
            fig.update_layout(xaxis_title='Presión Arterial Diastólica')
            fig.show()

            # Mostrar tabla descriptiva por grupo verticalmente
            tabla_agrupacion = datos.datos.groupby(agrupacion)['presion_arterial_diastolica'].describe().reset_index()
            tabla_agrupacion = tabla_agrupacion.rename(columns={'mean': 'Media', 'std': 'Desviación estándar', '50%': 'Mediana', 'min': 'Mínimo', 'max': 'Máximo'})
            tabla_agrupacion = tabla_agrupacion.melt(id_vars=agrupacion, var_name='Estadística', value_name='Valor')
            fig = go.Figure(data=[go.Table(
                header=dict(values=list(tabla_agrupacion.columns), fill_color='paleturquoise', align='left'),
                cells=dict(values=[tabla_agrupacion[col] for col in tabla_agrupacion.columns], fill_color='lavender', align='left')
            )])
            fig.update_layout(title_text=f'Estadísticas Descriptivas de presión arterial diastólica por {agrupacion}')
            fig.show()

    # Mostrar estadísticas descriptivas por intervalos
    datos.datos['presion_arterial_diastolica_intervalo'] = pd.cut(
        datos.datos['presion_arterial_diastolica'],
        intervalos['masculino'],  # Usar intervalo masculino como ejemplo
        right=False
    ).astype(str)
    tabla_intervalos = datos.datos.groupby('presion_arterial_diastolica_intervalo')['presion_arterial_diastolica'].describe().reset_index()
    tabla_intervalos.columns = ['Intervalo', 'Cuenta', 'Media', 'Desviación estándar', 'Mínimo', '25%', 'Mediana', '75%', 'Máximo']
    tabla_intervalos = tabla_intervalos.melt(id_vars='Intervalo', var_name='Estadística', value_name='Valor')
    fig = go.Figure(data=[go.Table(
        header=dict(values=list(tabla_intervalos.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[tabla_intervalos[col] for col in tabla_intervalos.columns], fill_color='lavender', align='left')
    )])
    fig.update_layout(title_text='Estadísticas Descriptivas por Intervalos de presión arterial diastólica')
    fig.show()
    intervalos_diastolica = [0, 80, 90, 100, float('inf')]
    labels_diastolica = ['[0, 80)', '[80, 90)', '[90, 100)', '[100, inf)']
    datos.datos['presion_arterial_diastolica_intervalo'] = pd.cut(
        datos.datos['presion_arterial_diastolica'],
        bins=intervalos_diastolica,
        labels=labels_diastolica,
        right=False,
        include_lowest=True
    )
    datos_filtrados_diastolica = datos.datos.dropna(subset=['presion_arterial_diastolica_intervalo'])
    datos_filtrados_diastolica['presion_arterial_diastolica_intervalo'] = pd.Categorical(
        datos_filtrados_diastolica['presion_arterial_diastolica_intervalo'],
        categories=labels_diastolica,
        ordered=True
    )
    fig = px.box(
        datos_filtrados_diastolica, 
        x='presion_arterial_diastolica_intervalo', 
        y='presion_arterial_diastolica', 
        title='Boxplot de Presión Arterial Diastólica por Intervalos', 
        color_discrete_sequence=["blue"]
    )
    fig.update_layout(
        xaxis_title='Intervalo', 
        yaxis_title='Presión Arterial Diastólica',
        xaxis={'categoryorder': 'array', 'categoryarray': labels_diastolica}  # Establecer el orden de los intervalos
    )
    fig.show()

plot_all()



C:\Users\loren\AppData\Local\Temp\ipykernel_17820\1731983727.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Conteos

In [71]:
for agrupacion in ['lugar_de_procedencia', 'lugar_de_nacimiento','tipo_de_afiliacion','escolaridad']:
            conteo_ciudades = datos.datos[agrupacion].value_counts().head(10)  
            df_ciudades = conteo_ciudades.reset_index()
            df_ciudades.columns = [agrupacion, 'Conteo']  
            fig = px.bar(df_ciudades, x=agrupacion, y='Conteo', title=f'10 Ciudades con Más Apariciones por {agrupacion}', 
                        color='Conteo', text='Conteo', color_continuous_scale='blues')
            fig.update_layout(xaxis_title=agrupacion, yaxis_title='Número de Apariciones', xaxis_tickangle=-45)
            fig.show()